In [129]:
import pandas as pd
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate, TimeSeriesSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from xgboost import XGBClassifier

In [2]:
!ls data_subset/

FS1.txt  profile.txt  PS2.txt


In [61]:
# Read each file into a DataFrame
df_PS2 = pd.read_csv("data_subset/PS2.txt", sep="\t", header=None)
df_FS1 = pd.read_csv("data_subset/FS1.txt", sep="\t", header=None)
df_profile = pd.read_csv("data_subset/profile.txt", sep="\t", header=None)

df_profile.columns = ['cooler_condition_%', 'valve_condition_%', 'internal_pump_leakage',
                      'hydraulic_accumulator_bar', 'stable_flag' ]

# # Set the headers as the lowercase name of the file
# df_PS2.columns = ['timestamp', 'pressure']
# df_FS1.columns = ['timestamp', 'volume_flow']
# df_profile.columns = ['timestamp', 'valve_condition']

In [49]:
df_profile.shape

(2205, 5)

In [50]:
df_PS2.shape

(2205, 6000)

In [51]:
df_FS1.shape

(2205, 600)

In [52]:
df_profile["valve_condition_%"].value_counts()

valve_condition_%
100    1125
73      360
80      360
90      360
Name: count, dtype: int64

In [53]:
df_FS1

,0,1,2,3,4,5,6,7,8,9,...,590,591,592,593,594,595,596,597,598,599
0,8.990,0.770,0.641,0.006,0.000,0.000,0.001,0.003,0.001,0.001,...,7.743,7.992,7.919,7.773,7.955,7.823,7.963,7.876,7.738,8.036
1,8.919,0.815,0.709,0.009,0.004,0.000,0.001,0.000,0.000,0.001,...,7.831,8.003,7.923,7.745,7.867,7.747,7.969,7.969,7.963,7.890
2,9.179,0.683,0.528,0.008,0.003,0.001,0.003,0.003,0.004,0.006,...,7.862,7.815,7.894,7.743,7.936,7.770,7.982,7.873,7.898,7.952
3,9.034,0.728,0.595,0.009,0.001,0.004,0.003,0.003,0.003,0.001,...,7.631,7.949,7.773,8.054,7.827,8.011,7.919,7.938,7.877,7.773
4,8.729,0.705,0.446,0.014,0.007,0.003,0.001,0.003,0.001,0.000,...,7.771,7.936,7.919,7.946,7.804,7.983,7.838,7.882,7.894,7.825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,8.173,1.024,0.834,0.009,0.000,0.001,0.000,0.003,0.001,0.000,...,7.904,7.777,7.938,7.841,7.741,8.004,7.869,7.719,8.003,7.874
2201,8.317,0.883,0.799,0.008,0.005,0.000,0.000,0.000,0.000,0.001,...,7.766,7.957,7.817,7.739,8.036,7.871,7.647,7.993,7.898,7.787
2202,8.170,0.686,0.583,0.011,0.000,0.003,0.004,0.000,0.000,0.000,...,7.898,7.648,7.953,7.871,7.859,7.921,7.770,7.982,7.859,7.685
2203,8.496,0.543,0.366,0.009,0.005,0.000,0.000,0.000,0.001,0.003,...,7.834,7.737,8.065,7.893,7.653,7.971,7.898,7.758,7.892,7.793


In [54]:
# # Transpose the FS1 DataFrame to have time series as rows
# df_FS1_transposed = df_FS1.T

# # Create a datetime index for the transposed DataFrame
# fs1_time_index = pd.date_range(start='2024-01-01', periods=len(df_FS1_transposed), freq='10ms')
# df_FS1_transposed.index = fs1_time_index

# # Resample FS1 to match the frequency of PS2 (from 10 Hz to 100 Hz)
# df_FS1_resampled_transposed = df_FS1_transposed.resample('10ms').interpolate()

# # Transpose back to the original format
# df_FS1_resampled = df_FS1_resampled_transposed.T

In [62]:
# Transpose the FS1 DataFrame to have time series as rows
df_FS1_transposed = df_FS1.T

# Create a datetime index for the transposed FS1 DataFrame
fs1_time_index = pd.date_range(start='2024-01-01', periods=len(df_FS1_transposed), freq='10ms')
df_FS1_transposed.index = fs1_time_index

# Resample FS1 to match the frequency of PS2 (from 10 Hz to 100 Hz)
df_FS1_resampled_transposed = df_FS1_transposed.resample('10ms').interpolate()

# Transpose back to the original format
df_FS1_resampled = df_FS1_resampled_transposed.T

# Transpose the PS2 DataFrame to have time series as rows
df_PS2_transposed = df_PS2.T

# Create a datetime index for the transposed PS2 DataFrame
ps2_time_index = pd.date_range(start='2024-01-01', periods=len(df_PS2_transposed), freq='10ms')
df_PS2_transposed.index = ps2_time_index

# Resample PS2 to match the frequency of FS1 (from 100 Hz to 10 Hz)
df_PS2_resampled_transposed = df_PS2_transposed.resample('100ms').interpolate()

# Transpose back to the original format
df_PS2_resampled = df_PS2_resampled_transposed.T



In [63]:
df_PS2_resampled

,2024-01-01 00:00:00.000,2024-01-01 00:00:00.100,2024-01-01 00:00:00.200,2024-01-01 00:00:00.300,2024-01-01 00:00:00.400,2024-01-01 00:00:00.500,2024-01-01 00:00:00.600,2024-01-01 00:00:00.700,2024-01-01 00:00:00.800,2024-01-01 00:00:00.900,...,2024-01-01 00:00:59.000,2024-01-01 00:00:59.100,2024-01-01 00:00:59.200,2024-01-01 00:00:59.300,2024-01-01 00:00:59.400,2024-01-01 00:00:59.500,2024-01-01 00:00:59.600,2024-01-01 00:00:59.700,2024-01-01 00:00:59.800,2024-01-01 00:00:59.900
0,125.50,15.648,6.266,3.250,2.008,1.367,1.078,0.836,0.641,0.578,...,125.32,125.48,125.19,125.26,125.21,125.26,125.22,125.18,125.34,125.02
1,125.06,18.531,7.250,3.414,2.187,1.492,1.125,0.836,0.680,0.602,...,125.02,125.06,124.95,125.26,125.00,124.92,125.08,125.18,125.10,124.80
2,125.13,16.125,6.117,2.914,1.914,1.305,0.914,0.633,0.547,0.375,...,124.84,124.72,124.71,124.82,124.72,124.62,124.75,124.83,124.84,124.61
3,124.93,18.328,5.156,2.250,1.352,0.758,0.656,0.500,0.375,0.211,...,124.47,124.38,124.81,124.73,124.48,124.69,124.61,124.73,124.49,124.82
4,124.72,20.555,7.266,3.266,1.953,1.227,1.078,0.797,0.562,0.477,...,124.39,124.38,124.41,124.59,124.39,124.69,124.69,124.59,124.47,124.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,125.47,21.297,5.289,1.984,1.102,0.758,0.555,0.234,0.141,0.078,...,125.53,125.63,125.91,125.69,125.57,125.79,125.84,125.77,125.55,125.57
2201,125.83,23.000,5.914,2.242,1.102,0.562,0.367,0.430,0.172,0.141,...,125.48,125.81,125.66,125.63,125.78,125.63,125.91,125.63,125.68,125.86
2202,125.41,19.805,5.016,1.953,1.023,0.602,0.484,0.422,0.289,0.117,...,125.81,125.83,125.49,125.78,125.86,125.51,125.71,125.99,125.53,125.66
2203,125.91,25.523,6.148,2.117,1.016,0.555,0.430,0.297,0.164,0.141,...,125.73,125.69,125.89,125.77,125.77,125.63,125.66,125.86,125.59,125.55


In [64]:
df_FS1_resampled

,2024-01-01 00:00:00.000,2024-01-01 00:00:00.010,2024-01-01 00:00:00.020,2024-01-01 00:00:00.030,2024-01-01 00:00:00.040,2024-01-01 00:00:00.050,2024-01-01 00:00:00.060,2024-01-01 00:00:00.070,2024-01-01 00:00:00.080,2024-01-01 00:00:00.090,...,2024-01-01 00:00:05.900,2024-01-01 00:00:05.910,2024-01-01 00:00:05.920,2024-01-01 00:00:05.930,2024-01-01 00:00:05.940,2024-01-01 00:00:05.950,2024-01-01 00:00:05.960,2024-01-01 00:00:05.970,2024-01-01 00:00:05.980,2024-01-01 00:00:05.990
0,8.990,0.770,0.641,0.006,0.000,0.000,0.001,0.003,0.001,0.001,...,7.743,7.992,7.919,7.773,7.955,7.823,7.963,7.876,7.738,8.036
1,8.919,0.815,0.709,0.009,0.004,0.000,0.001,0.000,0.000,0.001,...,7.831,8.003,7.923,7.745,7.867,7.747,7.969,7.969,7.963,7.890
2,9.179,0.683,0.528,0.008,0.003,0.001,0.003,0.003,0.004,0.006,...,7.862,7.815,7.894,7.743,7.936,7.770,7.982,7.873,7.898,7.952
3,9.034,0.728,0.595,0.009,0.001,0.004,0.003,0.003,0.003,0.001,...,7.631,7.949,7.773,8.054,7.827,8.011,7.919,7.938,7.877,7.773
4,8.729,0.705,0.446,0.014,0.007,0.003,0.001,0.003,0.001,0.000,...,7.771,7.936,7.919,7.946,7.804,7.983,7.838,7.882,7.894,7.825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,8.173,1.024,0.834,0.009,0.000,0.001,0.000,0.003,0.001,0.000,...,7.904,7.777,7.938,7.841,7.741,8.004,7.869,7.719,8.003,7.874
2201,8.317,0.883,0.799,0.008,0.005,0.000,0.000,0.000,0.000,0.001,...,7.766,7.957,7.817,7.739,8.036,7.871,7.647,7.993,7.898,7.787
2202,8.170,0.686,0.583,0.011,0.000,0.003,0.004,0.000,0.000,0.000,...,7.898,7.648,7.953,7.871,7.859,7.921,7.770,7.982,7.859,7.685
2203,8.496,0.543,0.366,0.009,0.005,0.000,0.000,0.000,0.001,0.003,...,7.834,7.737,8.065,7.893,7.653,7.971,7.898,7.758,7.892,7.793


In [81]:
def melt_dataframe(df):
    # Reset index to convert the timestamps to a regular column
    df = df.reset_index()
    
    # Melt the DataFrame to create the 'id', 'timestamp', and 'value' columns
    df_melted = pd.melt(df, id_vars=['index'], var_name='timestamp', value_name='value')
    
    # Rename the columns
    df_melted.columns = ['id', 'timestamp', 'value']
    
    # Sort by 'id' for better clarity
    df_melted = df_melted.sort_values(by='id').reset_index(drop=True)
    return df_melted

In [82]:
df_PS2_melted = melt_dataframe(df_PS2_resampled)

In [83]:
df_FS1_melted = melt_dataframe(df_FS1_resampled) #[lambda x: x.id==0]

In [84]:
df_PS2_melted

,id,timestamp,value
0,0,2024-01-01 00:00:00,125.50
1,0,2024-01-01 00:00:05.500000,0.00
2,0,2024-01-01 00:00:34.300000,140.41
3,0,2024-01-01 00:00:34.400000,140.60
4,0,2024-01-01 00:00:05.400000,0.00
...,...,...,...
1322995,2204,2024-01-01 00:00:39.700000,130.86
1322996,2204,2024-01-01 00:00:39.600000,130.93
1322997,2204,2024-01-01 00:00:39.500000,130.65
1322998,2204,2024-01-01 00:00:39.300000,131.94


In [87]:
df_FS1_melted[lambda x: x.id.isin([0,1,2,3,4])]

,id,timestamp,value
0,0,2024-01-01 00:00:00,8.990
1,0,2024-01-01 00:00:00.550000,0.000
2,0,2024-01-01 00:00:03.430000,7.773
3,0,2024-01-01 00:00:03.440000,7.968
4,0,2024-01-01 00:00:00.540000,0.001
...,...,...,...
2995,4,2024-01-01 00:00:02.110000,7.869
2996,4,2024-01-01 00:00:02.890000,7.995
2997,4,2024-01-01 00:00:02.270000,8.018
2998,4,2024-01-01 00:00:05.390000,7.881


In [101]:
df_FS1_melted[0:5]

,id,timestamp,value
0,0,2024-01-01 00:00:00,8.990
1,0,2024-01-01 00:00:00.550000,0.000
2,0,2024-01-01 00:00:03.430000,7.773
3,0,2024-01-01 00:00:03.440000,7.968
4,0,2024-01-01 00:00:00.540000,0.001


In [100]:
df_PS2_melted[0:5]

,id,timestamp,value
0,0,2024-01-01 00:00:00,125.50
1,0,2024-01-01 00:00:05.500000,0.00
2,0,2024-01-01 00:00:34.300000,140.41
3,0,2024-01-01 00:00:34.400000,140.60
4,0,2024-01-01 00:00:05.400000,0.00


In [105]:
df_full = pd.concat([df_FS1_melted.rename(columns={'value': 'fs1'}), df_PS2_melted.rename(columns={'value': 'ps2'})['ps2']], axis=1)
df_full

,id,timestamp,fs1,ps2
0,0,2024-01-01 00:00:00,8.990,125.50
1,0,2024-01-01 00:00:00.550000,0.000,0.00
2,0,2024-01-01 00:00:03.430000,7.773,140.41
3,0,2024-01-01 00:00:03.440000,7.968,140.60
4,0,2024-01-01 00:00:00.540000,0.001,0.00
...,...,...,...,...
1322995,2204,2024-01-01 00:00:03.970000,8.131,130.86
1322996,2204,2024-01-01 00:00:03.960000,7.908,130.93
1322997,2204,2024-01-01 00:00:03.950000,8.288,130.65
1322998,2204,2024-01-01 00:00:03.930000,9.444,131.94


In [114]:
#[lambda x: x.id.isin([0,1,2,3])]
df_extracted_features = extract_features(df_full, column_id="id", column_sort="timestamp", n_jobs=4)

Feature Extraction: 100%|█████████████████████████████████████████████████████████████████████████████| 20/20 [34:56<00:00, 104.84s/it]


In [116]:
# df_extracted_features.to_csv("./data/df_extracted_features.csv")

In [118]:
df_extracted_features.tail()

,ps2__variance_larger_than_standard_deviation,ps2__has_duplicate_max,ps2__has_duplicate_min,ps2__has_duplicate,ps2__sum_values,ps2__abs_energy,ps2__mean_abs_change,ps2__mean_change,ps2__mean_second_derivative_central,ps2__median,...,fs1__fourier_entropy__bins_5,fs1__fourier_entropy__bins_10,fs1__fourier_entropy__bins_100,fs1__permutation_entropy__dimension_3__tau_1,fs1__permutation_entropy__dimension_4__tau_1,fs1__permutation_entropy__dimension_5__tau_1,fs1__permutation_entropy__dimension_6__tau_1,fs1__permutation_entropy__dimension_7__tau_1,fs1__query_similarity_count__query_None__threshold_0.0,fs1__mean_n_absolute_max__number_of_maxima_7
2200,1.0,0.0,1.0,1.0,65876.820,8.573767e+06,0.743586,0.000167,0.087118,130.250,...,0.090729,0.200814,1.041474,1.702216,2.876840,3.979151,4.908310,5.581067,NaN,13.978857
2201,1.0,0.0,1.0,1.0,65881.801,8.574713e+06,0.743910,0.000050,0.086129,130.235,...,0.090729,0.200814,1.041474,1.732365,2.912586,4.024990,4.945603,5.565934,NaN,14.034857
2202,1.0,0.0,1.0,1.0,65864.583,8.571064e+06,0.745967,0.000417,0.088407,130.255,...,0.090729,0.200814,1.045530,1.734753,2.883012,3.898845,4.749061,5.378152,NaN,13.893429
2203,1.0,0.0,1.0,1.0,65884.231,8.574567e+06,0.747008,-0.000601,0.083902,130.155,...,0.090729,0.200814,1.041474,1.733812,2.939996,4.123328,5.035141,5.650461,NaN,13.994429
2204,1.0,0.0,1.0,1.0,65881.413,8.573638e+06,0.743005,-0.000668,0.083157,130.340,...,0.090729,0.200814,1.001631,1.694083,2.852960,3.904206,4.767004,5.397292,NaN,14.073143


In [127]:
df_to_model = pd.concat([df_extracted_features, df_profile["valve_condition_%"]], axis=1)
df_to_model['valve_condition_%'] = np.where(df_to_model['valve_condition_%'] == 100, 1, 0)
df_to_model

,ps2__variance_larger_than_standard_deviation,ps2__has_duplicate_max,ps2__has_duplicate_min,ps2__has_duplicate,ps2__sum_values,ps2__abs_energy,ps2__mean_abs_change,ps2__mean_change,ps2__mean_second_derivative_central,ps2__median,...,fs1__fourier_entropy__bins_10,fs1__fourier_entropy__bins_100,fs1__permutation_entropy__dimension_3__tau_1,fs1__permutation_entropy__dimension_4__tau_1,fs1__permutation_entropy__dimension_5__tau_1,fs1__permutation_entropy__dimension_6__tau_1,fs1__permutation_entropy__dimension_7__tau_1,fs1__query_similarity_count__query_None__threshold_0.0,fs1__mean_n_absolute_max__number_of_maxima_7,valve_condition_%
0,1.0,0.0,1.0,1.0,65684.199,8.523526e+06,0.672508,-0.000801,0.091582,129.300,...,0.228465,1.067784,1.722381,2.978515,4.232499,5.317004,5.937176,NaN,14.380714,1
1,1.0,0.0,1.0,1.0,65616.617,8.504898e+06,0.670334,-0.000434,0.088820,129.395,...,0.200814,1.055804,1.778340,3.066127,4.392241,5.460650,5.999139,NaN,13.844571,1
2,1.0,0.0,1.0,1.0,65501.551,8.476721e+06,0.671252,-0.000868,0.090949,129.305,...,0.200814,1.052981,1.768420,3.014061,4.363165,5.459621,6.023980,NaN,14.057143,1
3,1.0,0.0,1.0,1.0,65447.035,8.463371e+06,0.670761,-0.000184,0.089408,129.035,...,0.200814,1.086484,1.781593,2.965911,4.283595,5.388879,6.030677,NaN,14.330571,1
4,1.0,0.0,1.0,1.0,65365.587,8.440183e+06,0.680027,0.000134,0.087370,128.960,...,0.200814,0.988298,1.776346,2.990232,4.341014,5.426584,6.075373,NaN,13.948571,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,1.0,0.0,1.0,1.0,65876.820,8.573767e+06,0.743586,0.000167,0.087118,130.250,...,0.200814,1.041474,1.702216,2.876840,3.979151,4.908310,5.581067,NaN,13.978857,1
2201,1.0,0.0,1.0,1.0,65881.801,8.574713e+06,0.743910,0.000050,0.086129,130.235,...,0.200814,1.041474,1.732365,2.912586,4.024990,4.945603,5.565934,NaN,14.034857,1
2202,1.0,0.0,1.0,1.0,65864.583,8.571064e+06,0.745967,0.000417,0.088407,130.255,...,0.200814,1.045530,1.734753,2.883012,3.898845,4.749061,5.378152,NaN,13.893429,1
2203,1.0,0.0,1.0,1.0,65884.231,8.574567e+06,0.747008,-0.000601,0.083902,130.155,...,0.200814,1.041474,1.733812,2.939996,4.123328,5.035141,5.650461,NaN,13.994429,1


In [128]:
df_to_model["valve_condition_%"].value_counts()

valve_condition_%
1    1125
0    1080
Name: count, dtype: int64

In [143]:
X = df_to_model.drop(columns = ["valve_condition_%"])
y = df_to_model["valve_condition_%"].copy()
X_train, X_test = X.iloc[:500], X.iloc[500:]
y_train, y_test = y.iloc[:500], y.iloc[500:]

In [150]:
# Define the pipeline
pipeline = Pipeline([
    ('classifier', XGBClassifier(
        # random_state=42,
        # n_estimators=100,  # Number of trees in the ensemble
        # max_depth=2,  # Maximum depth of each tree
        # learning_rate=0.1,  # Step size shrinkage used in update to prevent overfitting
        # min_child_weight=1,  # Minimum sum of instance weight (hessian) needed in a child
        # subsample=0.8,  # Subsample ratio of the training instance
        # colsample_bytree=0.8,  # Subsample ratio of columns when constructing each tree
        # reg_alpha=0,  # L1 regularization term on weights
        # reg_lambda=1,  # L2 regularization term on weights
        # gamma=0,  # Minimum loss reduction required to make a further partition on a leaf node of the tree
    ))  # XGBoost classifier
])

# Define the evaluation metrics
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc'
}

# Perform time series cross-validation
cv_results = cross_validate(pipeline, X_train, y_train, cv=TimeSeriesSplit(n_splits=5), scoring=scoring)

# Compute relevant metrics
metrics_df = pd.DataFrame({
    'Accuracy': cv_results['test_accuracy'],
    'Precision': cv_results['test_precision'],
    'Recall': cv_results['test_recall'],
    'F1': cv_results['test_f1'],
    'AUC': cv_results['test_roc_auc']
})


/home/adelchiasta/venv/venv_vif_dataswati/lib64/python3.11/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
2 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "/home/adelchiasta/venv/venv_vif_dataswati/lib64/python3.11/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/adelchiasta/venv/venv_vif_dataswati/lib64/python3.11/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/adelchiasta/venv/v

In [152]:
metrics_df

,Accuracy,Precision,Recall,F1,AUC
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,0.867470,0.676471,1.000000,0.807018,1.000000
3,0.831325,0.909091,0.434783,0.588235,0.709058
4,0.987952,1.000000,0.966667,0.983051,0.983333


In [154]:
# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# Predict the target variable on the test data
y_pred = pipeline.predict(X_test)

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({
    'Accuracy': [accuracy],
    'Precision': [precision],
    'Recall': [recall],
    'F1': [f1],
    'AUC': [roc_auc]
})

# Print the metrics
print("Metrics on the test set:")
metrics_df

Metrics on the test set:


,Accuracy,Precision,Recall,F1,AUC
0,1.0,1.0,1.0,1.0,1.0


In [155]:
X_train

,ps2__variance_larger_than_standard_deviation,ps2__has_duplicate_max,ps2__has_duplicate_min,ps2__has_duplicate,ps2__sum_values,ps2__abs_energy,ps2__mean_abs_change,ps2__mean_change,ps2__mean_second_derivative_central,ps2__median,...,fs1__fourier_entropy__bins_5,fs1__fourier_entropy__bins_10,fs1__fourier_entropy__bins_100,fs1__permutation_entropy__dimension_3__tau_1,fs1__permutation_entropy__dimension_4__tau_1,fs1__permutation_entropy__dimension_5__tau_1,fs1__permutation_entropy__dimension_6__tau_1,fs1__permutation_entropy__dimension_7__tau_1,fs1__query_similarity_count__query_None__threshold_0.0,fs1__mean_n_absolute_max__number_of_maxima_7
0,1.0,0.0,1.0,1.0,65684.199,8.523526e+06,0.672508,-0.000801,0.091582,129.300,...,0.090729,0.228465,1.067784,1.722381,2.978515,4.232499,5.317004,5.937176,NaN,14.380714
1,1.0,0.0,1.0,1.0,65616.617,8.504898e+06,0.670334,-0.000434,0.088820,129.395,...,0.090729,0.200814,1.055804,1.778340,3.066127,4.392241,5.460650,5.999139,NaN,13.844571
2,1.0,0.0,1.0,1.0,65501.551,8.476721e+06,0.671252,-0.000868,0.090949,129.305,...,0.090729,0.200814,1.052981,1.768420,3.014061,4.363165,5.459621,6.023980,NaN,14.057143
3,1.0,0.0,1.0,1.0,65447.035,8.463371e+06,0.670761,-0.000184,0.089408,129.035,...,0.090729,0.200814,1.086484,1.781593,2.965911,4.283595,5.388879,6.030677,NaN,14.330571
4,1.0,0.0,1.0,1.0,65365.587,8.440183e+06,0.680027,0.000134,0.087370,128.960,...,0.090729,0.200814,0.988298,1.776346,2.990232,4.341014,5.426584,6.075373,NaN,13.948571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1.0,0.0,1.0,1.0,62738.122,7.808618e+06,0.637679,-0.000117,0.083564,124.490,...,0.090729,0.254093,1.235780,1.719274,2.946121,4.273212,5.350884,5.987376,NaN,15.162000
496,1.0,0.0,1.0,1.0,73564.808,1.097007e+07,0.782407,-0.000033,0.080003,132.340,...,0.125256,0.170467,0.783563,1.757014,3.074456,4.502504,5.659176,6.182292,NaN,16.833000
497,1.0,0.0,1.0,1.0,73663.901,1.097953e+07,0.761753,0.000017,0.079919,132.700,...,0.125256,0.170467,0.815007,1.761761,3.080884,4.496508,5.675742,6.178768,NaN,17.341571
498,1.0,0.0,1.0,1.0,73670.311,1.098314e+07,0.781302,-0.000401,0.083214,133.155,...,0.125256,0.170467,0.837917,1.763062,3.077774,4.510885,5.675954,6.168670,NaN,17.476857
